### Chapter 1. 파이썬 데이터 모델

C++ stl 같은 경우, 대부분 str.length() 나 vector.size() 같은 메서드를 이용해서 길이를 구한다. 그러나 파이썬은 특이하게 len(seq) 를 이용해서 인수의 길이를 구한다. 파이썬에는 이러한 특이한 점들이 여러 가지 있고, 이것을 이해하는 것이 파이썬을 이해하고 또 '파이써닉' 하다는 것을 이해할 수 있는 발판이 된다.

그럼 파이썬은 왜 len() 을 쓰는가? 물론 len 메서드가 존재하지 않는 것은 아니다. `__len__` 이라는 메서드가 있어서, 실제로 len() 함수가 `__len__` 을 호출할 때도 있다. 그러나 그렇게 단순하게 돌아가지만은 않는다. 가령 다음과 같은 코드를 보자.

In [ ]:
class foo:
    word="Python"
    def __len__(self):
        print("__len__ method called")
        return "this is string"

obj=foo()
print(obj.__len__())
print(len(obj))

위의 코드를 실행시켜 보면, 분명 `__len__` 을 직접 호출하나 `len()` 을 사용하나 `__len__` 이 호출된다는 것을 알 수 있다. 하지만 `len()`을 사용할 경우 위 코드는 에러가 발생한다. 길이를 구하는 함수인데 문자열을 리턴하도록 함수를 짰기 때문이다. `len()`을 사용할 시에는 의 결과값이 정수형인지 한 번 검사해 주는 것이다. 이런 식으로, `len()` 은 단순한 편리 구문이 아니다.

또한 `len()` 의 내부 구현이 꼭 `__len__` 의 호출로만 이루어지지 않을 때도 있다. 사용자 정의 클래스를 사용할 시에는 거의 `__len__` 이 호출되지만, 리스트 등 내부 자료형의 길이를 구하는 CPython 함수는 내부적으로 저장해 둔 길이 변수(PyVarObject의 ob_size 필드값)를 리턴해 주는 것이다. 이는 시간복잡도 면에서 매우 큰 이득이다.

이런 많은 것들을 뭉쳐서 파이썬에서는 len() 함수 하나로 만들었다. 이는 알아보기도 쉽고, 어떤 반복형의 길이를 구한다는 것을 직관적으로 알 수 있는 작명이라고 한다. 귀도 반 로섬의 파이썬 메일링 리스트에도 나와 있다(https://mail.python.org/pipermail/python-3000/2006-November/004643.html)

이런 걸 적절하게 이해하는 것이 파이썬 철학 이해의 핵심이다.

또 하나의 예시는 `__len__` 과 `__getitem__` 만 있으면 반복, 슬라이싱 등이 가능해지는 것이다.

In [ ]:
class alpha: #알파벳 대문자를 담은 연습용 클래스
    letters=[chr(i) for i in range(65,91)]
    def __len__(self):
        return len(self.letters)

    def __getitem__(self, pos):
        return self.letters[pos]



foo=alpha()
print(len(foo)) #알파벳은 26개이다
print(foo[3]) #알파벳 리스트의 4번째를 가져오므로 D

print('D' in foo) #검색 가능
print(foo[:3]) #슬라이싱 가능
for l in foo: #반복 가능
    print(l)

26
D
True
['A', 'B', 'C']
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
U
V
W
X
Y
Z


len() 과 인덱싱이 작동하는 건 해당 메서드를 구현했으니 당연하다. 하지만 슬라이싱과 검색, 반복이 작동한다는 것은 신기한 일이다. 물론 검색을 하는 `__contains__` 와 반복을 담당하는 `__iter__` 메서드가 있으나, `__len__` 과 `__getitem__` 만으로도 대신할 수도 있다. 이는 프로토콜과 관련이 있는데 책의 뒤 내용에서 자세하게 다룬다. 

아무튼 이런 식으로 특별 메서드를 사용하는 데이터 모델은 많은 이점이 있다. 읽기도 쉬울 뿐더러 내장 데이터형에서 구현한 함수 이름, 형식들을 사용자 정의 클래스에도 사용할 수 있게 하는 것도 큰 이점이고, 파이썬에서 이미 강력하게 구현해 놓은 데이터 모델을 그대로 사용할 수 있다는 것도 장점이다.
(이를테면 `__contains__` 가 구현되어 있지 않을 시에 `__getitem__` 을 이용해서 순차적으로 검색한다든지 하는 기능을 파이썬에서 기본적으로 제공한다. 이런 건 사용자가 직접 만들기 쉽지 않다)

또한 내장 자료형의 특별 메서드를 직접 호출하는 게 아니라 아까의 `__len__` 처럼 어떤 다른 내부 구현이 있을 수도 있다. 이는 메서드 호출보다 빠르기 때문에 쓰는 것이 좋다. 이런 내부 자료형의 함수명 등을 사용자 정의 클래스에서도 쓸 수 있는 건 프로그래머 입장에서 편하다.

단 주의할 점으로, `__func__` 와 같이 이중 언더바가 붙은 함수 작명은 파이썬의 특별 메서드와 헷갈릴 여지가 매우 크기 때문에 함수 작명시에 쓰지 말아야 한다는 것을 기억해 두자.

특별 메서드의 예시로 간단한 벡터 클래스를 구현한다.

In [ ]:
from math import hypot

class Vector:
    def __init__(self, x=0, y=0):
        self.x=x
        self.y=y

    def __repr__(self): #객체의 문자열 표현
        return 'Vector(%r, %r)' % (self.x, self.y)

    def __abs__(self): #절댓값(벡터 길이)
        return hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))
        #만약에 __bool__ 이 구현되어 있지 않으면 파이썬은 __len__을 호출해서 길이가 0이면 False, 1이면 True를 반환한다. 여기선 bool을 구현했다.

    def __add__(self, other): #연산자 오버로딩
        x=self.x+other.x
        y=self.y+other.y
        return Vector(x,y)

    def __mul__(self, scalar):
        return Vector(self.x*scalar, self.y*scalar)
        #여기서 문제는, 이렇게 구현할 시 벡터에 스칼라를 곱하는 건 가능하지만 스칼라에 벡터를 곱하는 건 안된다는 것이다.
        #이는 교환법칙의 위배인데. __rmul__ 을 이용할 경우 해결 가능하다.

a=Vector(1,2)
b=Vector(3,4)
print(a+b)

Vector(4, 6)
